In [ ]:
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sqlite3

con = sqlite3.connect('output/database.sqlite')
data = pd.read_sql("SELECT *  \
FROM Scorecard \
WHERE 1 = 1 \
AND md_earn_wne_p6 != 'PrivacySuppressed' AND md_earn_wne_p6 IS NOT NULL \
", con)
#Year = 2011



In [ ]:
cols_target = ['md_earn_wne_p6']

cols_school = ['PREDDEG', 'HIGHDEG', 'CONTROL', 'NUMBRANCH', 'AVGFACSAL']
#                'PCIP01', 'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10',
#                'PCIP11', 'PCIP12', 'PCIP13', 'PCIP14', 'PCIP15', 'PCIP16',
#                'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24', 'PCIP25', 'PCIP26',
#                'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38', 'PCIP39',
#                'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44', 'PCIP45',
#                'PCIP46', 'PCIP47', 'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51',
#                'PCIP52', 'PCIP54']

# cols_admissions = ['ADM_RATE', 'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'SAT_AVG']
cols_admissions = ['ADM_RATE', 'SAT_AVG']


cols_costs = ['TUITFTE']

cols_studentbody = ['UGDS', 'UGDS_NRA', 'PPTUG_EF', 'UG25abv',
                    'PAR_ED_PCT_1STGEN', 'DEP_INC_AVG', 'IND_INC_AVG',
                    'COMP_ORIG_YR2_RT', 'WDRAW_ORIG_YR2_RT', 'ENRL_ORIG_YR2_RT',
                    'COMP_ORIG_YR4_RT', 'WDRAW_ORIG_YR4_RT', 'ENRL_ORIG_YR4_RT',
                    'OVERALL_YR2_N', 'OVERALL_YR3_N', 'OVERALL_YR4_N',
                    'OVERALL_YR6_N', 'OVERALL_YR8_N', 'count_nwne_p6']

cols_financialaid = ['DEBT_MDN', 'GRAD_DEBT_MDN', 'WDRAW_DEBT_MDN']

cols_other = []
#['type']


    
#data_reduced = data[cols_target+cols_school+cols_admissions+cols_costs+\
#                    cols_studentbody+cols_financialaid+cols_other]

data_reduced = data[data.describe().columns]


In [ ]:
data_reduced[(data_reduced.CONTROL=='Private for-profit') ]


In [ ]:
# explore response variable for lineriatry
plt.figure()
#sns.distplot(data_reduced[cols_target])
sns.distplot(data_reduced[data.CONTROL=='Public'][cols_target], label ='Public')
sns.distplot(data_reduced[(data.CONTROL=='Private nonprofit')][cols_target], label='Private nonprofit')
sns.distplot(data_reduced[(data.CONTROL=='Private for-profit')][cols_target] , label= 'Private for-profit')



In [ ]:
sns.boxplot(x="CONTROL", y="md_earn_wne_p6", data=data,
            order=np.sort(data.CONTROL.unique()), color='White',
            fliersize=0, width=0.25)
sns.stripplot(x="CONTROL", y="md_earn_wne_p6", data=data,
              order=np.sort(data.CONTROL.unique()),
              alpha=0.25, size=5,
              color='#348ABD', edgecolor='#348ABD')

In [ ]:
#Step 1 PCA
#Scale the features and center 
from sklearn import preprocessing
#X = data[cols_admissions+cols_costs+\
#                    cols_studentbody+cols_financialaid+cols_other]

X = data[data.describe().columns]

X.dropna(inplace=True)

X_scaled = preprocessing.scale(X)
y = data[cols_target].values

print pd.DataFrame(X_scaled).shape
print y.shape

Recursive feature elimination

Recursive feature elimination is based on the idea to repeatedly construct a model (for example an SVM or a regression model) and choose either the best or worst performing feature (for example based on coefficients), setting the feature aside and then repeating the process with the rest of the features. This process is applied until all features in the dataset are exhausted. Features are then ranked according to when they were eliminated. As such, it is a greedy optimization for finding the best performing subset of features.

The stability of RFE depends heavily on the type of model that is used for feature ranking at each iteration. Just as non-regularized regression can be unstable, so can RFE when utilizing it, while using ridge regression can provide more stable results.

Sklearn provides RFE for recursive feature elimination and RFECV for finding the ranks together with optimal number of features via a cross validation loop.

In [ ]:
##http://blog.datadive.net/selecting-good-features-part-ii-linear-models-and-regularization/

from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso, RandomizedLasso)
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from minepy import MINE
 
np.random.seed(0)
 

X = X_scaled
 
 
names = cols_admissions+cols_costs+\
                    cols_studentbody+cols_financialaid+cols_other

 
ranks = {}
 
def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))
 
lr = LinearRegression(normalize=True)
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X_scaled, y)
ranks["Linear reg"] = rank_to_dict(map(lambda x: round(x, 4), rfe.ranking_) , names)

ridge = Ridge(alpha=0.1)
rfe = RFE(ridge, n_features_to_select=1)
rfe.fit(X_scaled,y)
ranks["Ridge"] = rank_to_dict(map(lambda x: round(x, 4), rfe.ranking_) , names)

 

lasso = Lasso(alpha=.05)
lasso.fit(X_scaled,y)
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)
 
 

#stop the search when 5 features are left (they will get equal scores)

rfe = RFE(lr, n_features_to_select=5)
rfe.fit(X_scaled,y)
ranks["RFE"] = rank_to_dict(map(float, rfe.ranking_), names, order=-1)
 
rlasso = RandomizedLasso(alpha=0.04)
rlasso.fit(X_scaled, y)
ranks["Stability"] = rank_to_dict(np.abs(rlasso.scores_), names)


rf = RandomForestRegressor()
rf.fit(X_scaled,y)
ranks["RF"] = rank_to_dict(rf.feature_importances_, names)

f, pval  = f_regression(X_scaled, y, center=True)
ranks["Corr."] = rank_to_dict(f, names)


mine = MINE()
mic_scores = []
#for i in range(X.shape[1]):
     #mine.compute_score(X[:,i], y)
     #m = mine.mic()
     #mic_scores.append(m)
 
#ranks["MIC"] = rank_to_dict(mic_scores, names) 
 
 
r = {}
for name in names:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)

methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")
 
print "\t%s" % "\t".join(methods)
for name in names:
    print "%s\t%s" % (name, "\t".join(map(str, 
                         [ranks[method][name] for method in methods])))
 



To Evaluate variables I created a matrix of regressors and messured fature imortance.

* Corr 
F-regression just computes the F statistic and pick the best features. F-regression does the following:
- Start with a constant model, M0
- Try all models M1 consisting of just one feature and pick the best according to the F statistic
- Try all models M2 consisting of M1 plus one other feature and pick the best ...

*  Recursive Feature Elimination (RFE) Recursive feature elimination is based on the idea to repeatedly construct a model (for example an SVM or a regression model) and choose either the best or worst performing feature (for example based on coefficients), setting the feature aside and then repeating the process with the rest of the features. This process is applied until all features in the dataset are exhausted. Features are then ranked according to when they were eliminated. As such, it is a greedy optimization for finding the best performing subset of features.

Additionally I look at 

- Linear Regression
- Ridge Regression
- Randomized Lasso 
- Random Forest 


 

From the list of columns and its scores  I see some interesting obesvations:
- Stability. Using Ridge regression for data interpretation due to its stability and the fact that useful features tend to have non-zero coefficients. 

Following features have low stability ( coefficients close to 0) and can be removed:
* Percentage of undergraduates aged 25 and above	UG25abv
* Number of students in overall 3-year completion cohort	OVERALL_YR3_N

In previous tests I observed that Random Forests have the best predictive power messured by Median SQ Error MSE. 
Using Random Forest Feature Importances I can see tha following Features are most important:
Average family income of dependent students in real 2014 dollars.	 DEP_INC_AVG <br>
Percent who transferred to a 4-year institution and completed within 4 years	COMP_ORIG_YR4_RT <br>
Average family income of independent students in real 2015 dollars.	IND_INC_AVG <br>
Average SAT equivalent score of students admitted	SAT_AVG <br>
Percent still enrolled at original institution within 2 years	ENRL_ORIG_YR2_RT <br>
Enrollment of undergraduate certificate/degree-seeking students	UGDS <br>
Percent withdrawn from original institution within 2 years WDRAW_ORIG_YR2_RT <br>
The median debt for students who have completed	GRAD_DEBT_MDN <br>
Percent completed within 2 years at original institution	COMP_ORIG_YR2_RT <br>
Percentage first-generation students	 PAR_ED_PCT_1STGEN <br>
Tuition revenue per full-time equivalent student	TUITFTE <br>
